In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from io import StringIO # python3; python2: BytesIO 
import boto3
import s3fs
import awscli
import timeit

In [2]:
tic=timeit.default_timer()
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

5.3600000001097214e-05

In [3]:
s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

team3-final-bucket
wshih-bucket


In [4]:
my_bucket = s3.Bucket('team3-final-bucket')
for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object.key)

Resources_clean/
Resources_clean/acquisitions_clean.csv
Resources_clean/degrees_clean.csv
Resources_clean/funding_rounds_clean.csv
Resources_clean/funding_rounds_final.csv
Resources_clean/funds_clean.csv
Resources_clean/investments_clean.csv
Resources_clean/ipo_clean.csv
Resources_clean/milestones_clean.csv
Resources_clean/objects_clean.csv
Resources_clean/offices_clean.csv
Resources_clean/people_clean.csv
Resources_clean/relationships_clean.csv
Resources_raw/
Resources_raw/acquisitions.csv
Resources_raw/degrees.csv
Resources_raw/funding_rounds.csv
Resources_raw/funds.csv
Resources_raw/investments.csv
Resources_raw/ipos.csv
Resources_raw/milestones.csv
Resources_raw/objects.csv
Resources_raw/offices.csv
Resources_raw/people.csv
Resources_raw/relationships.csv
sql_db/
sql_db/acquisitions_clean.csv
sql_db/degrees_clean.csv
sql_db/funding_round_filled.csv
sql_db/funds_clean.csv
sql_db/investments_cleaner.csv
sql_db/ipos_cleaner.csv
sql_db/milestones_cleaner.csv
sql_db/objects_clean.csv
sq

######  Just for creating a layout I will be temporary loading local csv files as some of them I have downloaded on my laptop only

In [5]:
# Loading the file with compiled start dates
companies_df = pd.read_csv('companies_info.csv')
companies_df

,object_id,entity_type,name_x,category_code,status,country_code,state_code,city,region,investment_rounds,invested_companies,funding_rounds,funding_total_usd
0,c:1,Company,Wetpaint,web,operating,USA,WA,Seattle,Seattle,0,0,3,39750000
1,c:10,Company,Flektor,games_video,acquired,USA,CA,Culver City,Los Angeles,0,0,0,0
2,c:100,Company,There,games_video,acquired,USA,CA,San Mateo,SF Bay,0,0,0,0
3,c:10000,Company,MYWEBBO,network_hosting,operating,NaN,NaN,NaN,unknown,0,0,0,0
4,c:10001,Company,THE Movie Streamer,games_video,operating,NaN,NaN,NaN,unknown,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,Webgility,ecommerce,operating,USA,CA,San Francisco,SF Bay,0,0,0,0
196549,c:9995,Company,FohBoh,public_relations,operating,USA,CA,San Jose,SF Bay,0,0,1,750000
196550,c:9996,Company,CSC,consulting,operating,USA,VA,Falls Church,Washington DC,0,0,0,0
196551,c:9997,Company,Top-candidate,search,operating,USA,CA,Los Angeles,Los Angeles,0,0,0,0


In [7]:
companies_df.columns.to_list()

['object_id',
 'entity_type',
 'name_x',
 'category_code',
 'status',
 'country_code',
 'state_code',
 'city',
 'region',
 'investment_rounds',
 'invested_companies',
 'funding_rounds',
 'funding_total_usd']

In [9]:
companies_info = companies_df[['object_id',
 'category_code',
 'status',
 'funding_total_usd']]
companies_info

,object_id,category_code,status,funding_total_usd
0,c:1,web,operating,39750000
1,c:10,games_video,acquired,0
2,c:100,games_video,acquired,0
3,c:10000,network_hosting,operating,0
4,c:10001,games_video,operating,0
...,...,...,...,...
196548,c:99940,ecommerce,operating,0
196549,c:9995,public_relations,operating,750000
196550,c:9996,consulting,operating,0
196551,c:9997,search,operating,0


In [10]:
# Loading the file with product labels
labels_df = pd.read_csv('prod_label.csv')
labels_df

,object_id,product_label
0,c:1,1
1,c:10,1
2,c:100,0
3,c:10000,0
4,c:10001,0
...,...,...
196548,c:99940,1
196549,c:9995,0
196550,c:9996,0
196551,c:9997,0


In [11]:
# Merging them
companies_prodlbls = pd.merge(companies_info, labels_df, how='left', on=['object_id'])
companies_prodlbls

,object_id,category_code,status,funding_total_usd,product_label
0,c:1,web,operating,39750000,1
1,c:10,games_video,acquired,0,1
2,c:100,games_video,acquired,0,0
3,c:10000,network_hosting,operating,0,0
4,c:10001,games_video,operating,0,0
...,...,...,...,...,...
196548,c:99940,ecommerce,operating,0,1
196549,c:9995,public_relations,operating,750000,0
196550,c:9996,consulting,operating,0,0
196551,c:9997,search,operating,0,0


In [12]:
# Loading funding rounds types, # of rounds and # of participants
funds_df = pd.read_csv('companies_funding_rounds.csv')
funds_df

,object_id,rounds_total,last_funding_round_type,number_of_participants
0,c:1,3.0,series-c+,9.0
1,c:10,NaN,NaN,NaN
2,c:100,NaN,NaN,NaN
3,c:10000,NaN,NaN,NaN
4,c:10001,NaN,NaN,NaN
...,...,...,...,...
196548,c:99940,NaN,NaN,NaN
196549,c:9995,1.0,angel,0.0
196550,c:9996,NaN,NaN,NaN
196551,c:9997,NaN,NaN,NaN


In [13]:
# Merging them
companies_lbls_funds = pd.merge(companies_prodlbls, funds_df, how='left', on=['object_id'])
companies_lbls_funds

,object_id,category_code,status,funding_total_usd,product_label,rounds_total,last_funding_round_type,number_of_participants
0,c:1,web,operating,39750000,1,3.0,series-c+,9.0
1,c:10,games_video,acquired,0,1,NaN,NaN,NaN
2,c:100,games_video,acquired,0,0,NaN,NaN,NaN
3,c:10000,network_hosting,operating,0,0,NaN,NaN,NaN
4,c:10001,games_video,operating,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
196548,c:99940,ecommerce,operating,0,1,NaN,NaN,NaN
196549,c:9995,public_relations,operating,750000,0,1.0,angel,0.0
196550,c:9996,consulting,operating,0,0,NaN,NaN,NaN
196551,c:9997,search,operating,0,0,NaN,NaN,NaN


In [14]:
companies_lbls_funds.columns.to_list()

['object_id',
 'category_code',
 'status',
 'funding_total_usd',
 'product_label',
 'rounds_total',
 'last_funding_round_type',
 'number_of_participants']

In [15]:
# Loading final file with all date compiled based on Fahima's file
#client = boto3.client('s3')
#path1 = 's3://team3-final-bucket/'
time_elapsed_df = pd.read_csv('companies_all_dates.csv')
time_elapsed_df

,object_id,start_dates,end_dates,time_elapsed_months
0,c:1,2005-10-01,2008-05-19,32.0
1,c:10,2011-09-01,NaN,NaN
2,c:100,NaN,NaN,NaN
3,c:10000,2008-07-26,2015-01-01,77.0
4,c:10001,2008-07-26,2015-01-01,77.0
...,...,...,...,...
196548,c:99940,2007-01-01,2015-01-01,96.0
196549,c:9995,2007-11-01,2008-08-25,10.0
196550,c:9996,1959-01-01,2015-01-01,672.0
196551,c:9997,2008-07-01,2015-01-01,78.0


In [16]:
# Merging companies and dates
companies_with_dates = pd.merge(companies_lbls_funds, time_elapsed_df, how='left', on=['object_id'])
companies_with_dates

,object_id,category_code,status,funding_total_usd,product_label,rounds_total,last_funding_round_type,number_of_participants,start_dates,end_dates,time_elapsed_months
0,c:1,web,operating,39750000,1,3.0,series-c+,9.0,2005-10-01,2008-05-19,32.0
1,c:10,games_video,acquired,0,1,NaN,NaN,NaN,2011-09-01,NaN,NaN
2,c:100,games_video,acquired,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,c:10000,network_hosting,operating,0,0,NaN,NaN,NaN,2008-07-26,2015-01-01,77.0
4,c:10001,games_video,operating,0,0,NaN,NaN,NaN,2008-07-26,2015-01-01,77.0
...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,ecommerce,operating,0,1,NaN,NaN,NaN,2007-01-01,2015-01-01,96.0
196549,c:9995,public_relations,operating,750000,0,1.0,angel,0.0,2007-11-01,2008-08-25,10.0
196550,c:9996,consulting,operating,0,0,NaN,NaN,NaN,1959-01-01,2015-01-01,672.0
196551,c:9997,search,operating,0,0,NaN,NaN,NaN,2008-07-01,2015-01-01,78.0


In [17]:
# Loading Winston's file
locations = pd.read_csv('usa_only.csv')
locations

,Unnamed: 0,object_id,name,address1,address2,city,zip_code,state_code,country_code,latitude,longitude,region
0,0,c:1,Wetpaint,710 - 2nd Avenue,Suite 1100,Seattle,98104,WA,USA,47.603122,-122.333253,West
1,6454,c:15710,Immersion Corporation,801 Fox Lane,San Jose 95131,SF,0,CA,USA,37.386711,-121.903953,NorCal
2,10539,c:17385,About Me!,"2601 Mission St,","#900,",SF,94110,CA,USA,37.755068,-122.418026,NorCal
3,3,c:10005,Thomas Publishing,Five Penn Plaza,15th Floor,New York,10001,NY,USA,0.000000,0.000000,NY
4,4,c:10009,dimension5 labs,1025 Don Diego Ave,NaN,Santa Fe,87505,NM,USA,35.674675,-105.948434,Southwest
...,...,...,...,...,...,...,...,...,...,...,...,...
55468,44595,c:65540,The Pricerie,NaN,NaN,LA,0,CA,USA,0.000000,0.000000,SoCal
55469,48079,c:75652,Textual Indices,NaN,NaN,SF,95382,CA,USA,0.000000,0.000000,NorCal
55470,55599,f:9997,Axel Johnson,155 Spring Street,6th floor,New York,10012,NY,USA,40.724468,-74.001704,NY
55471,55600,f:9998,Liberty City Ventures,NaN,NaN,New York,0,NY,USA,40.714353,-74.005973,NY


In [18]:
# Dropping columns
locations_clean = locations[['object_id','country_code','region']]

In [19]:
# Merging companies and locations
companies_locations = pd.merge(companies_with_dates, locations_clean, how='left', on=['object_id'])
companies_locations

,object_id,category_code,status,funding_total_usd,product_label,rounds_total,last_funding_round_type,number_of_participants,start_dates,end_dates,time_elapsed_months,country_code,region
0,c:1,web,operating,39750000,1,3.0,series-c+,9.0,2005-10-01,2008-05-19,32.0,USA,West
1,c:10,games_video,acquired,0,1,NaN,NaN,NaN,2011-09-01,NaN,NaN,USA,SoCal
2,c:100,games_video,acquired,0,0,NaN,NaN,NaN,NaN,NaN,NaN,USA,NorCal
3,c:10000,network_hosting,operating,0,0,NaN,NaN,NaN,2008-07-26,2015-01-01,77.0,NaN,NaN
4,c:10001,games_video,operating,0,0,NaN,NaN,NaN,2008-07-26,2015-01-01,77.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,ecommerce,operating,0,1,NaN,NaN,NaN,2007-01-01,2015-01-01,96.0,USA,NorCal
196549,c:9995,public_relations,operating,750000,0,1.0,angel,0.0,2007-11-01,2008-08-25,10.0,USA,NorCal
196550,c:9996,consulting,operating,0,0,NaN,NaN,NaN,1959-01-01,2015-01-01,672.0,USA,Northeast
196551,c:9997,search,operating,0,0,NaN,NaN,NaN,2008-07-01,2015-01-01,78.0,USA,SoCal


In [20]:
# We need only USA companies so filtering them
USA_companies = companies_locations[companies_locations['country_code']=='USA']
USA_companies

,object_id,category_code,status,funding_total_usd,product_label,rounds_total,last_funding_round_type,number_of_participants,start_dates,end_dates,time_elapsed_months,country_code,region
0,c:1,web,operating,39750000,1,3.0,series-c+,9.0,2005-10-01,2008-05-19,32.0,USA,West
1,c:10,games_video,acquired,0,1,NaN,NaN,NaN,2011-09-01,NaN,NaN,USA,SoCal
2,c:100,games_video,acquired,0,0,NaN,NaN,NaN,NaN,NaN,NaN,USA,NorCal
8,c:10005,advertising,operating,0,0,NaN,NaN,NaN,NaN,2015-01-01,NaN,USA,NY
12,c:10009,advertising,operating,0,1,NaN,NaN,NaN,2008-08-01,2015-01-01,77.0,USA,Southwest
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,ecommerce,operating,0,1,NaN,NaN,NaN,2007-01-01,2015-01-01,96.0,USA,NorCal
196549,c:9995,public_relations,operating,750000,0,1.0,angel,0.0,2007-11-01,2008-08-25,10.0,USA,NorCal
196550,c:9996,consulting,operating,0,0,NaN,NaN,NaN,1959-01-01,2015-01-01,672.0,USA,Northeast
196551,c:9997,search,operating,0,0,NaN,NaN,NaN,2008-07-01,2015-01-01,78.0,USA,SoCal


In [22]:
# Checking null values
for column in USA_companies.columns:
    print(f"Column {column} has {USA_companies[column].isnull().sum()} null values")

Column object_id has 0 null values
Column category_code has 4305 null values
Column status has 0 null values
Column funding_total_usd has 0 null values
Column product_label has 0 null values
Column rounds_total has 31260 null values
Column last_funding_round_type has 31842 null values
Column number_of_participants has 31842 null values
Column start_dates has 12831 null values
Column end_dates has 7232 null values
Column time_elapsed_months has 17782 null values
Column country_code has 0 null values
Column region has 26 null values


In [23]:
# Filtering companies only from 2000 year
USA_companies1 = USA_companies[USA_companies['start_dates']>='2000-01-01']
USA_companies1

,object_id,category_code,status,funding_total_usd,product_label,rounds_total,last_funding_round_type,number_of_participants,start_dates,end_dates,time_elapsed_months,country_code,region
0,c:1,web,operating,39750000,1,3.0,series-c+,9.0,2005-10-01,2008-05-19,32.0,USA,West
1,c:10,games_video,acquired,0,1,NaN,NaN,NaN,2011-09-01,NaN,NaN,USA,SoCal
12,c:10009,advertising,operating,0,1,NaN,NaN,NaN,2008-08-01,2015-01-01,77.0,USA,Southwest
13,c:1001,web,acquired,5000000,1,1.0,series-a,3.0,2007-10-01,NaN,NaN,USA,NorCal
15,c:10011,games_video,operating,0,1,NaN,NaN,NaN,2008-01-01,2015-01-01,84.0,USA,SoCal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196545,c:99938,software,operating,0,0,NaN,NaN,NaN,2004-01-01,2015-01-01,132.0,USA,Northeast
196548,c:99940,ecommerce,operating,0,1,NaN,NaN,NaN,2007-01-01,2015-01-01,96.0,USA,NorCal
196549,c:9995,public_relations,operating,750000,0,1.0,angel,0.0,2007-11-01,2008-08-25,10.0,USA,NorCal
196551,c:9997,search,operating,0,0,NaN,NaN,NaN,2008-07-01,2015-01-01,78.0,USA,SoCal


In [116]:
USA_companies1['status'].unique()

array(['operating', 'acquired', 'closed', 'ipo'], dtype=object)

In [118]:
USA_companies1[USA_companies1['status']=='acquired'].count()

object_id                  2157
category_code              2011
status                     2157
funding_total_usd          2157
product_label              2157
rounds_total               1306
last_funding_round_type    1296
number_of_participants     1296
start_dates                2157
end_dates                     0
time_elapsed_months           0
country_code               2157
region                     2157
dtype: int64

In [24]:
# Checking null values
for column in USA_companies1.columns:
    print(f"Column {column} has {USA_companies1[column].isnull().sum()} null values")

Column object_id has 0 null values
Column category_code has 869 null values
Column status has 0 null values
Column funding_total_usd has 0 null values
Column product_label has 0 null values
Column rounds_total has 17626 null values
Column last_funding_round_type has 17751 null values
Column number_of_participants has 17751 null values
Column start_dates has 0 null values
Column end_dates has 3349 null values
Column time_elapsed_months has 3349 null values
Column country_code has 0 null values
Column region has 16 null values


In [120]:
# Filtering out missing end_dates values
# USA_companies2 = USA_companies1[USA_companies1['end_dates'].notnull()]
# USA_companies2

In [121]:
# USA_companies2['status'].unique()

In [122]:
# Checking null values
# for column in USA_companies2.columns:
    # print(f"Column {column} has {USA_companies2[column].isnull().sum()} null values")

In [123]:
# Dropping nulls in category_code
USA_companies3 = USA_companies1[USA_companies1['category_code'].notnull()]
USA_companies3

,object_id,category_code,status,funding_total_usd,product_label,rounds_total,last_funding_round_type,number_of_participants,start_dates,end_dates,time_elapsed_months,country_code,region
0,c:1,web,operating,39750000,1,3.0,series-c+,9.0,2005-10-01,2008-05-19,32.0,USA,West
1,c:10,games_video,acquired,0,1,NaN,NaN,NaN,2011-09-01,NaN,NaN,USA,SoCal
12,c:10009,advertising,operating,0,1,NaN,NaN,NaN,2008-08-01,2015-01-01,77.0,USA,Southwest
13,c:1001,web,acquired,5000000,1,1.0,series-a,3.0,2007-10-01,NaN,NaN,USA,NorCal
15,c:10011,games_video,operating,0,1,NaN,NaN,NaN,2008-01-01,2015-01-01,84.0,USA,SoCal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196545,c:99938,software,operating,0,0,NaN,NaN,NaN,2004-01-01,2015-01-01,132.0,USA,Northeast
196548,c:99940,ecommerce,operating,0,1,NaN,NaN,NaN,2007-01-01,2015-01-01,96.0,USA,NorCal
196549,c:9995,public_relations,operating,750000,0,1.0,angel,0.0,2007-11-01,2008-08-25,10.0,USA,NorCal
196551,c:9997,search,operating,0,0,NaN,NaN,NaN,2008-07-01,2015-01-01,78.0,USA,SoCal


In [125]:
# Dropping rows with negative number of months in time_elapsed
#USA_companies4 = USA_companies3[USA_companies1['time_elapsed_months']>=0]
#USA_companies4

In [126]:
# Checking null values
#for column in USA_companies4.columns:
   # print(f"Column {column} has {USA_companies4[column].isnull().sum()} null values")

In [127]:
# Dropping missing values in regions
USA_companies5 = USA_companies3[USA_companies1['region'].notnull()].reset_index(drop=True)
USA_companies5

C:\Users\hp\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,object_id,category_code,status,funding_total_usd,product_label,rounds_total,last_funding_round_type,number_of_participants,start_dates,end_dates,time_elapsed_months,country_code,region
0,c:1,web,operating,39750000,1,3.0,series-c+,9.0,2005-10-01,2008-05-19,32.0,USA,West
1,c:10,games_video,acquired,0,1,NaN,NaN,NaN,2011-09-01,NaN,NaN,USA,SoCal
2,c:10009,advertising,operating,0,1,NaN,NaN,NaN,2008-08-01,2015-01-01,77.0,USA,Southwest
3,c:1001,web,acquired,5000000,1,1.0,series-a,3.0,2007-10-01,NaN,NaN,USA,NorCal
4,c:10011,games_video,operating,0,1,NaN,NaN,NaN,2008-01-01,2015-01-01,84.0,USA,SoCal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30966,c:99938,software,operating,0,0,NaN,NaN,NaN,2004-01-01,2015-01-01,132.0,USA,Northeast
30967,c:99940,ecommerce,operating,0,1,NaN,NaN,NaN,2007-01-01,2015-01-01,96.0,USA,NorCal
30968,c:9995,public_relations,operating,750000,0,1.0,angel,0.0,2007-11-01,2008-08-25,10.0,USA,NorCal
30969,c:9997,search,operating,0,0,NaN,NaN,NaN,2008-07-01,2015-01-01,78.0,USA,SoCal


##### Now we need to bucket companies life span

In [128]:
time_elapsed_months = USA_companies5['time_elapsed_months']
max_val = time_elapsed_months.max()
max_val

180.0

In [129]:
time_bins = [-0.01, 6, 12, 24, 36, 60, 180]
months_in_bins = pd.cut(USA_companies5['time_elapsed_months'], time_bins)
months_in_bins

0         (24.0, 36.0]
1                  NaN
2        (60.0, 180.0]
3                  NaN
4        (60.0, 180.0]
             ...      
30966    (60.0, 180.0]
30967    (60.0, 180.0]
30968      (6.0, 12.0]
30969    (60.0, 180.0]
30970      (6.0, 12.0]
Name: time_elapsed_months, Length: 30971, dtype: category
Categories (6, interval[float64]): [(-0.01, 6.0] < (6.0, 12.0] < (12.0, 24.0] < (24.0, 36.0] < (36.0, 60.0] < (60.0, 180.0]]

In [130]:
# Naming the groups:
group_names = ['0-6mth','6-12mth','1-2yr','2-3yr','3-5yr','over5yr']

In [131]:
USA_companies6 = USA_companies5.copy()
USA_companies6['time_range'] = pd.cut(USA_companies5['time_elapsed_months'], time_bins, labels=group_names)
USA_companies6

,object_id,category_code,status,funding_total_usd,product_label,rounds_total,last_funding_round_type,number_of_participants,start_dates,end_dates,time_elapsed_months,country_code,region,time_range
0,c:1,web,operating,39750000,1,3.0,series-c+,9.0,2005-10-01,2008-05-19,32.0,USA,West,2-3yr
1,c:10,games_video,acquired,0,1,NaN,NaN,NaN,2011-09-01,NaN,NaN,USA,SoCal,NaN
2,c:10009,advertising,operating,0,1,NaN,NaN,NaN,2008-08-01,2015-01-01,77.0,USA,Southwest,over5yr
3,c:1001,web,acquired,5000000,1,1.0,series-a,3.0,2007-10-01,NaN,NaN,USA,NorCal,NaN
4,c:10011,games_video,operating,0,1,NaN,NaN,NaN,2008-01-01,2015-01-01,84.0,USA,SoCal,over5yr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30966,c:99938,software,operating,0,0,NaN,NaN,NaN,2004-01-01,2015-01-01,132.0,USA,Northeast,over5yr
30967,c:99940,ecommerce,operating,0,1,NaN,NaN,NaN,2007-01-01,2015-01-01,96.0,USA,NorCal,over5yr
30968,c:9995,public_relations,operating,750000,0,1.0,angel,0.0,2007-11-01,2008-08-25,10.0,USA,NorCal,6-12mth
30969,c:9997,search,operating,0,0,NaN,NaN,NaN,2008-07-01,2015-01-01,78.0,USA,SoCal,over5yr


In [84]:
# USA_companies6.to_csv('USA_companies6_new.csv', index=False)

In [132]:
USA_companies6.dtypes

object_id                    object
category_code                object
status                       object
funding_total_usd             int64
product_label                 int64
rounds_total                float64
last_funding_round_type      object
number_of_participants      float64
start_dates                  object
end_dates                    object
time_elapsed_months         float64
country_code                 object
region                       object
time_range                 category
dtype: object

In [133]:
USA_companies6['time_range'] = USA_companies6['time_range'].astype('object')

In [134]:
USA_companies6.dtypes

object_id                   object
category_code               object
status                      object
funding_total_usd            int64
product_label                int64
rounds_total               float64
last_funding_round_type     object
number_of_participants     float64
start_dates                 object
end_dates                   object
time_elapsed_months        float64
country_code                object
region                      object
time_range                  object
dtype: object

In [135]:
USA_companies6['time_range'] = USA_companies6['time_range'].replace(np.nan, 'unknown')

##### Selecting features


In [136]:
USA_companies6.columns.to_list()

['object_id',
 'category_code',
 'status',
 'funding_total_usd',
 'product_label',
 'rounds_total',
 'last_funding_round_type',
 'number_of_participants',
 'start_dates',
 'end_dates',
 'time_elapsed_months',
 'country_code',
 'region',
 'time_range']

In [137]:
USA_companies7 = USA_companies6[['object_id',
 'category_code',
 'status',
 'product_label',
 'rounds_total',
 'number_of_participants',
 'region',
 'time_range']]
USA_companies7

,object_id,category_code,status,product_label,rounds_total,number_of_participants,region,time_range
0,c:1,web,operating,1,3.0,9.0,West,2-3yr
1,c:10,games_video,acquired,1,NaN,NaN,SoCal,unknown
2,c:10009,advertising,operating,1,NaN,NaN,Southwest,over5yr
3,c:1001,web,acquired,1,1.0,3.0,NorCal,unknown
4,c:10011,games_video,operating,1,NaN,NaN,SoCal,over5yr
...,...,...,...,...,...,...,...,...
30966,c:99938,software,operating,0,NaN,NaN,Northeast,over5yr
30967,c:99940,ecommerce,operating,1,NaN,NaN,NorCal,over5yr
30968,c:9995,public_relations,operating,0,1.0,0.0,NorCal,6-12mth
30969,c:9997,search,operating,0,NaN,NaN,SoCal,over5yr


In [138]:
# Replace NaNs with 0 in a few columns - 
# funding_rounds_count, number_of_participants,time_elapsed_months 
USA_companies7['rounds_total'] = USA_companies7['rounds_total'].replace(np.nan, 0)
USA_companies7['number_of_participants'] = USA_companies7['number_of_participants'].replace(np.nan, 0)
USA_companies7

C:\Users\hp\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,object_id,category_code,status,product_label,rounds_total,number_of_participants,region,time_range
0,c:1,web,operating,1,3.0,9.0,West,2-3yr
1,c:10,games_video,acquired,1,0.0,0.0,SoCal,unknown
2,c:10009,advertising,operating,1,0.0,0.0,Southwest,over5yr
3,c:1001,web,acquired,1,1.0,3.0,NorCal,unknown
4,c:10011,games_video,operating,1,0.0,0.0,SoCal,over5yr
...,...,...,...,...,...,...,...,...
30966,c:99938,software,operating,0,0.0,0.0,Northeast,over5yr
30967,c:99940,ecommerce,operating,1,0.0,0.0,NorCal,over5yr
30968,c:9995,public_relations,operating,0,1.0,0.0,NorCal,6-12mth
30969,c:9997,search,operating,0,0.0,0.0,SoCal,over5yr


In [139]:
USA_companies7.columns.to_list()

['object_id',
 'category_code',
 'status',
 'product_label',
 'rounds_total',
 'number_of_participants',
 'region',
 'time_range']

In [92]:
# USA_companies7.to_csv('USA_companies_to_train.csv', sep=',', index=False)

# ML part

###### Splitting the table to categorical and numerical features


In [140]:
# Categorical features
categorical = USA_companies7[[
 'category_code',
 'region',
 'time_range']]
categorical

,category_code,region,time_range
0,web,West,2-3yr
1,games_video,SoCal,unknown
2,advertising,Southwest,over5yr
3,web,NorCal,unknown
4,games_video,SoCal,over5yr
...,...,...,...
30966,software,Northeast,over5yr
30967,ecommerce,NorCal,over5yr
30968,public_relations,NorCal,6-12mth
30969,search,SoCal,over5yr


In [141]:
categorical.dtypes

category_code    object
region           object
time_range       object
dtype: object

In [142]:
# Checking null values
for column in categorical.columns:
    print(f"Column {column} has {categorical[column].isnull().sum()} null values")

Column category_code has 0 null values
Column region has 0 null values
Column time_range has 0 null values


### First we would want to create a correlation matrix to find dependant valiables

In [143]:
# Encoding features
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [144]:
categorical_encode = categorical.copy()

In [145]:
categorical_encode['category_code'] = le.fit_transform(categorical_encode['category_code'])
categorical_encode['region'] = le.fit_transform(categorical_encode['region'])
categorical_encode['time_range'] = le.fit_transform(categorical_encode['time_range'])
categorical_encode

,category_code,region,time_range
0,41,7,2
1,12,4,6
2,0,6,5
3,41,2,6
4,12,4,5
...,...,...,...
30966,37,3,5
30967,7,2,5
30968,31,2,4
30969,33,4,5


In [146]:
# Numerical features
numerical = USA_companies7[['product_label',
 'rounds_total',
 'number_of_participants']]
numerical

,product_label,rounds_total,number_of_participants
0,1,3.0,9.0
1,1,0.0,0.0
2,1,0.0,0.0
3,1,1.0,3.0
4,1,0.0,0.0
...,...,...,...
30966,0,0.0,0.0
30967,1,0.0,0.0
30968,0,1.0,0.0
30969,0,0.0,0.0


In [147]:
# Merging dfs to combine all features and dropping IDs
features_corr = pd.merge(categorical_encode, numerical, how='left', left_index=True, right_index=True)
features_corr

,category_code,region,time_range,product_label,rounds_total,number_of_participants
0,41,7,2,1,3.0,9.0
1,12,4,6,1,0.0,0.0
2,0,6,5,1,0.0,0.0
3,41,2,6,1,1.0,3.0
4,12,4,5,1,0.0,0.0
...,...,...,...,...,...,...
30966,37,3,5,0,0.0,0.0
30967,7,2,5,1,0.0,0.0
30968,31,2,4,0,1.0,0.0
30969,33,4,5,0,0.0,0.0


##### Creating a correlation matrix to find if there are dependent variables

In [148]:
corrMatrix = features_corr.corr()
corrMatrix.style.background_gradient(cmap='coolwarm')

,category_code,region,time_range,product_label,rounds_total,number_of_participants
category_code,1.000000,0.001227,0.024239,0.080485,-0.047789,-0.067083
region,0.001227,1.000000,0.042950,-0.021705,-0.013745,-0.076559
time_range,0.024239,0.042950,1.000000,0.123129,0.002617,0.030952
product_label,0.080485,-0.021705,0.123129,1.000000,0.041079,0.097607
rounds_total,-0.047789,-0.013745,0.002617,0.041079,1.000000,0.276202
number_of_participants,-0.067083,-0.076559,0.030952,0.097607,0.276202,1.000000


There are no dependant variables

In [165]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix,classification_report
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.metrics import classification_report_imbalanced
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN
from sklearn.decomposition import PCA

In [149]:
USA_companies7

,object_id,category_code,status,product_label,rounds_total,number_of_participants,region,time_range
0,c:1,web,operating,1,3.0,9.0,West,2-3yr
1,c:10,games_video,acquired,1,0.0,0.0,SoCal,unknown
2,c:10009,advertising,operating,1,0.0,0.0,Southwest,over5yr
3,c:1001,web,acquired,1,1.0,3.0,NorCal,unknown
4,c:10011,games_video,operating,1,0.0,0.0,SoCal,over5yr
...,...,...,...,...,...,...,...,...
30966,c:99938,software,operating,0,0.0,0.0,Northeast,over5yr
30967,c:99940,ecommerce,operating,1,0.0,0.0,NorCal,over5yr
30968,c:9995,public_relations,operating,0,1.0,0.0,NorCal,6-12mth
30969,c:9997,search,operating,0,0.0,0.0,SoCal,over5yr


In [150]:
# Defining target
у_df = USA_companies7[['object_id','status']]
у_df

,object_id,status
0,c:1,operating
1,c:10,acquired
2,c:10009,operating
3,c:1001,acquired
4,c:10011,operating
...,...,...
30966,c:99938,operating
30967,c:99940,operating
30968,c:9995,operating
30969,c:9997,operating


In [151]:
у_df['status'].unique()

array(['operating', 'acquired', 'closed', 'ipo'], dtype=object)

In [152]:
# Setting labels in the column for our target (acqusition)
у1_df_copy = у_df.copy()
у1_df_copy['status'].replace(to_replace=['acquired'], value='0', inplace=True)
у1_df_copy['status'].replace(to_replace=['operating', 'closed', 'ipo'], value='1', inplace=True)
у1_df_copy

,object_id,status
0,c:1,1
1,c:10,0
2,c:10009,1
3,c:1001,0
4,c:10011,1
...,...,...
30966,c:99938,1
30967,c:99940,1
30968,c:9995,1
30969,c:9997,1


In [153]:
у1_df_copy.dtypes

object_id    object
status       object
dtype: object

In [154]:
у1_df_copy.groupby('status')['status'].count()

status
0     2011
1    28960
Name: status, dtype: int64

In [159]:
# Defining the features for scaling
X_df = pd.merge(categorical, numerical, left_index=True, right_index=True)
X_df

,category_code,region,time_range,product_label,rounds_total,number_of_participants
0,web,West,2-3yr,1,3.0,9.0
1,games_video,SoCal,unknown,1,0.0,0.0
2,advertising,Southwest,over5yr,1,0.0,0.0
3,web,NorCal,unknown,1,1.0,3.0
4,games_video,SoCal,over5yr,1,0.0,0.0
...,...,...,...,...,...,...
30966,software,Northeast,over5yr,0,0.0,0.0
30967,ecommerce,NorCal,over5yr,1,0.0,0.0
30968,public_relations,NorCal,6-12mth,0,1.0,0.0
30969,search,SoCal,over5yr,0,0.0,0.0


In [161]:
# Encode the categorical features
X_encoded = pd.get_dummies(X_df, columns=['category_code',
 'region',
 'time_range'])
X_encoded

,product_label,rounds_total,number_of_participants,category_code_advertising,category_code_analytics,category_code_automotive,category_code_biotech,category_code_cleantech,category_code_consulting,category_code_design,...,region_South,region_Southwest,region_West,time_range_0-6mth,time_range_1-2yr,time_range_2-3yr,time_range_3-5yr,time_range_6-12mth,time_range_over5yr,time_range_unknown
0,1,3.0,9.0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,1,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0.0,0.0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,1,1.0,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30966,0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30967,1,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30968,0,1.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
30969,0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [162]:
y1 = у1_df_copy['status']
y1

0        1
1        0
2        1
3        0
4        1
        ..
30966    1
30967    1
30968    1
30969    1
30970    1
Name: status, Length: 30971, dtype: object

In [166]:
# Splitting the data to train / test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y1, random_state=1, stratify=y1, test_size=0.25)

In [167]:
# We need to scale some features
data_scaler = StandardScaler()
X_train_scaled = data_scaler.fit_transform(X_train)
X_test_scaled = data_scaler.fit_transform(X_test)

In [168]:
# Count target train data
Counter(y_train)

Counter({'1': 21720, '0': 1508})

In [185]:
# Resampling with SMOTE Oversampling
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({'1': 21720, '0': 21720})

#### Training

##### Random Forest

In [170]:
# Creating random forest classifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

In [171]:
# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

In [186]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [188]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)
cm

array([[ 489,   14],
       [ 291, 6949]], dtype=int64)

In [189]:
# Out of confusion matrix making a df with proper label names
df_rand_for = pd.DataFrame(data=cm, columns=["predicted_Acquisition","predicted_non_Acquisition"], index=["actual_Acquisition","actual_non_Acquisition"])
df_rand_for

,predicted_Acquisition,predicted_non_Acquisition
actual_Acquisition,489,14
actual_non_Acquisition,291,6949


In [190]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.960609582849025

In [191]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.63      0.97      0.76       503
           1       1.00      0.96      0.98      7240

    accuracy                           0.96      7743
   macro avg       0.81      0.97      0.87      7743
weighted avg       0.97      0.96      0.96      7743



The accuracy score is very high but if we want to know which company we would acquire then we should focus on precision. Our result is 63% and could be improved.

To improve our model we need to find the features that impact the outcome. For that we will use linear regression.

### Applying linear regression to select features - Backward Elimination Technique

In [177]:
from sklearn.linear_model import LinearRegression
import statsmodels.regression.linear_model as sm

In [178]:
from scipy import stats
import numpy as np

In [212]:
# Splitting the data to train / test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y1, random_state=1, stratify=y1, test_size=0.25)

In [215]:
# We need to scale some features because we have to fit it to a training model
data_scaler = StandardScaler()
X_train_scaled = data_scaler.fit_transform(X_train)
X_test_scaled = data_scaler.fit_transform(X_test)

In [220]:
# Count target train data
Counter(y_train)

Counter({'1': 21720, '0': 1508})

In [221]:
# Resampling with SMOTE Oversampling
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({'1': 21720, '0': 21720})

In [222]:
lm = LinearRegression() 
lm.fit(X_resampled, y_resampled) 
 

LinearRegression()

In [223]:
#pred = lm.predict(X_test_scaled)
print(lm.score(X_test_scaled, y_test))

-3.609118699002682e+20


In [225]:
X_resampled = np.append (arr=np.ones([X_resampled.shape[0],1]).astype(int), values = X_resampled, axis = 1)

In [234]:
x_opt = [0, 1, 2, 3, 4, 5,6,7,8,9,10,1,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60] 


In [235]:
regressor = sm.OLS(y_resampled.astype(float), X_resampled[:,x_opt].astype(float)).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                 status   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                     574.2
Date:                Sun, 09 Aug 2020   Prob (F-statistic):               0.00
Time:                        09:32:06   Log-Likelihood:                -19484.
No. Observations:               43440   AIC:                         3.908e+04
Df Residuals:                   43383   BIC:                         3.958e+04
Df Model:                          56                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6853      0.002    327.532      0.0